In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Đọc dữ liệu từ các file CSV trong thư mục Test
ratings_path = "/content/drive/MyDrive/Test/ratings.csv"
movies_path = "/content/drive/MyDrive/Test/movies.csv"

In [4]:
ratings = pd.read_csv(ratings_path)
movies = pd.read_csv(movies_path)

In [5]:
ratings_sample = ratings.sample(n=50, random_state=42)

In [6]:
print(ratings_sample)

          userId  movieId  rating   timestamp
10685861   66954      781     5.0   850944577
1552723     9877      574     4.0   945495614
6145184    38348     1088     2.0   999974867
16268584  101952     2706     1.0  1203077565
22418634  140400   275079     3.5  1653782463
27638726  173400     1079     4.0  1133884396
11890228   74417    69526     1.0  1511831626
31139054  195523     2762     5.0  1196244339
306095      1953     7451     4.0  1117552226
13213609   82682     1210     5.0  1525844054
14864676   93112    30793     4.0  1168789318
4637050    29008     7024     0.5  1580407766
2374514    15092    68157     4.5  1266293957
10248899   64197    48516     3.5  1211725237
31461429  197624    55247     5.0  1675683191
21804088  136388      784     3.0  1482780322
15811578   99083     1721     2.0  1310890388
13581399   84937     1210     5.0  1051378880
3949478    24772     1378     4.0  1067220789
25979212  163220    53887     3.5  1589879484
11947306   74771   174055     4.5 

In [7]:
top_users = ratings_sample['userId'].value_counts().nlargest(1000).index
top_movies = ratings_sample['movieId'].value_counts().nlargest(1000).index

In [8]:
filtered_ratings = ratings_sample[ratings_sample['userId'].isin(top_users) & ratings_sample['movieId'].isin(top_movies)]

In [9]:
merged_df = pd.merge(movies, filtered_ratings, on='movieId')

In [10]:
users = merged_df['userId'].unique()
movies = merged_df['movieId'].unique()
ratings_matrix = pd.DataFrame(index=users, columns=movies)

In [11]:
# Điền giá trị ratings vào ma trận
for _, row in merged_df.iterrows():
    ratings_matrix.loc[row['userId'], row['movieId']] = row['rating']

# Thay NaN bằng giá trị ngẫu nhiên trong khoảng [1, 5]
ratings_matrix = ratings_matrix.applymap(lambda x: random.uniform(1, 5) if pd.isna(x) else x)


<ipython-input-11-751cd84ca753>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ratings_matrix = ratings_matrix.applymap(lambda x: random.uniform(1, 5) if pd.isna(x) else x)


In [13]:
# Cập nhật mẫu Gibbs Sampling với phân phối có điều kiện (Gaussian hoặc khác)
def update_gibbs_sample(ratings_matrix, users, movies):
    for user in users:
        for movie in movies:
            # Lấy các giá trị rating khác của người dùng đối với các bộ phim khác
            others = ratings_matrix.loc[user, [col for col in movies if col != movie]]
            mean_rating = others.mean()  # Trung bình các rating của người dùng đối với các phim khác
            std_rating = others.std() if others.std() > 0 else 1  # Đảm bảo phương sai lớn hơn 0

            # Lấy mẫu từ phân phối Gaussian có điều kiện
            new_rating = np.random.normal(mean_rating, std_rating)

            # Cập nhật giá trị rating trong ma trận
            ratings_matrix.loc[user, movie] = max(1, min(5, new_rating))

# Gibbs Sampling
def gibbs_sample(ratings_matrix, users, movies, num_iterations):
    for _ in range(num_iterations):
        update_gibbs_sample(ratings_matrix, users, movies)


In [14]:
# Tạo mẫu Gibbs
num_iterations = 100
gibbs_sample(ratings_matrix, users, movies, num_iterations)

In [15]:
# Dự đoán rating
predicted_ratings = []
predicted_results = []

In [16]:
# Lấy rating dự đoán từ ma trận ratings_matrix
for _, row in ratings_sample.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']

    # Dự đoán rating từ ma trận đã được cập nhật qua Gibbs Sampling
    predicted_rating = ratings_matrix.loc[user_id, movie_id]
    predicted_ratings.append(predicted_rating)

    # Lưu cặp (userId, movieId) và rating dự đoán để in
    predicted_results.append({
        'userId': user_id,
        'movieId': movie_id,
        'predicted_rating': predicted_rating
    })

# Chuyển kết quả dự đoán thành DataFrame
predicted_df = pd.DataFrame(predicted_results)

In [17]:
# In kết quả rating dự đoán ra màn hình
print("Predicted Ratings for users and movies:")
print(predicted_df)


Predicted Ratings for users and movies:
      userId   movieId  predicted_rating
0    66954.0     781.0          2.870846
1     9877.0     574.0          2.618179
2    38348.0    1088.0          1.599358
3   101952.0    2706.0          3.678717
4   140400.0  275079.0          3.887370
5   173400.0    1079.0          2.756731
6    74417.0   69526.0          3.017697
7   195523.0    2762.0          2.539266
8     1953.0    7451.0          2.522786
9    82682.0    1210.0          3.459141
10   93112.0   30793.0          3.818512
11   29008.0    7024.0          3.792858
12   15092.0   68157.0          3.275642
13   64197.0   48516.0          4.329411
14  197624.0   55247.0          2.812048
15  136388.0     784.0          2.668534
16   99083.0    1721.0          3.743322
17   84937.0    1210.0          2.580480
18   24772.0    1378.0          2.987250
19  163220.0   53887.0          2.373119
20   74771.0  174055.0          4.134089
21  175222.0   91500.0          3.303224
22    9268.0    8

In [18]:
# Lọc các giá trị rating thực tế (y_i)
true_ratings = []
for _, row in ratings_sample.iterrows():
    true_ratings.append(row['rating'])

In [19]:
rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
print(f"RMSE: {rmse}")

RMSE: 1.4941207114492308
